<h1 align=center><font size = 5>Regression Models with Keras</font></h1>
<h1 align=center><font size = 5>by</font></h1>
<h1 align=center><font size = 5>Md. Anas Mondol</font></h1>

## Download and Clean Dataset

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Anaconda Jupyter Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's read the data into a <em>pandas</em> dataframe.

In [3]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [4]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to n_cols since we will need this number when building our network.

In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

## Import Keras

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [13]:
from keras.models import Sequential
from keras.layers import Dense

## A. Build a Baseline Model

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

In [14]:
# Create the neural network in a function so we can use it multiple times in the
# subsequent sections

def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function creates a model that has one hidden layer with 10 neurons and a ReLU activation function. It uses the adam optimizer and the mean squared error as the loss function.

__1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the 
`train_test_split` helper function from Scikit-learn.__

Let's import scikit-learn in order to randomly split the data into a training and test sets

In [15]:
from sklearn.model_selection import train_test_split

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


Splitting the data into a training and test sets by holding 30% of the data for testing

In [16]:
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size=0.3, random_state=71)
# let's have a look at the shape of the predictors set
predictors_train.shape

(721, 8)

Let's call the function now to create our model.

In [17]:
# build the model
model = regression_model()

__2. Train the model on the training data using `50 epochs`.__

In [21]:
model.fit(predictors_train, target_train, epochs=50, verbose=2)

Epoch 1/50
 - 0s - loss: 103.8261
Epoch 2/50
 - 0s - loss: 103.7494
Epoch 3/50
 - 0s - loss: 103.8028
Epoch 4/50
 - 0s - loss: 106.0420
Epoch 5/50
 - 0s - loss: 109.6676
Epoch 6/50
 - 0s - loss: 104.5277
Epoch 7/50
 - 0s - loss: 104.5742
Epoch 8/50
 - 0s - loss: 105.6978
Epoch 9/50
 - 0s - loss: 108.5148
Epoch 10/50
 - 0s - loss: 102.0741
Epoch 11/50
 - 0s - loss: 102.9474
Epoch 12/50
 - 0s - loss: 103.1287
Epoch 13/50
 - 0s - loss: 103.8133
Epoch 14/50
 - 0s - loss: 105.5895
Epoch 15/50
 - 0s - loss: 104.3058
Epoch 16/50
 - 0s - loss: 105.4239
Epoch 17/50
 - 0s - loss: 103.4057
Epoch 18/50
 - 0s - loss: 103.1668
Epoch 19/50
 - 0s - loss: 106.8940
Epoch 20/50
 - 0s - loss: 103.1724
Epoch 21/50
 - 0s - loss: 103.2326
Epoch 22/50
 - 0s - loss: 102.5841
Epoch 23/50
 - 0s - loss: 103.0866
Epoch 24/50
 - 0s - loss: 105.0978
Epoch 25/50
 - 0s - loss: 102.9596
Epoch 26/50
 - 0s - loss: 102.3090
Epoch 27/50
 - 0s - loss: 102.1616
Epoch 28/50
 - 0s - loss: 103.5480
Epoch 29/50
 - 0s - loss: 102

__3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the `mean_squared_error` function from Scikit-learn.__

In [24]:
predictions_test = model.predict(predictors_test)

from sklearn.metrics import mean_squared_error
print('Mean Squared Error on test data is: %.3f' % (mean_squared_error(target_test, predictions_test)))

Mean Squared Error on test data is: 104.532


__4. Repeat steps 1 - 3, `50` times, i.e., create a list of `50` mean squared errors.__

__5. Report the `mean and the standard deviation of the mean squared errors`.__

In [26]:
# Create a function that evaluates the model so we can use it to evaluate the models created in part A, B, C and D
# the 'create_model_func' parameter is the function that is used to build the model. For part A, this is the
# regression_model function defined above

def evaluate_model(create_model_func, predictors, targets, epochs=50):
    mean_squared_errors = []
    
    for i in range(50):
        # create the model. I wasn't 100% clear whether this should be inside the loop, but I _think_ that was the
        # intent of the question. Otherwise, the average and stddev of the mean squared error is not that meaningful
        model = create_model_func()
        
        # 1. split the data in a train and test set
        predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, 
                                                                                    test_size=0.3, random_state=71)
        # 2. train 50 epochs (suppress logging this time)
        model.fit(predictors_train, target_train, epochs=epochs, verbose=0)
        
        # 3. measure the mse and add this to the list
        predictions_test = model.predict(predictors_test)
        mse = mean_squared_error(target_test, predictions_test)
        mean_squared_errors.append(mse)
        print('.', end='') # output a dot so we can see that the function is still running
    print(' Done!')
    # return the mean and stddev of the mse list
    return np.mean(mean_squared_errors), np.std(mean_squared_errors)

In [27]:
# Evaluate the model and print the mean and std dev of the mean squared errors. Note that we pass in 
# the regression_model _function_ here. This is used in the evaluate_model function to create a fresh
# neural network in each loop

mean_mse, std_mse = evaluate_model(regression_model, predictors, target)

# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models: mean = %.3f, std dev = %.3f" %(mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models: mean = 301.392, std dev = 273.568


## B. Normalize the data 

Repeat Part A __but use a normalized version of the data__. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [28]:
mean_mse, std_mse = evaluate_model(regression_model, predictors_norm, target)
# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models on normalized data: mean = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models on normalized data: mean = 314.919, std dev = 57.559


__How does the mean of the mean squared errors compare to that from Step A?__

The mean of the mean squared error has gone up a bit (~5%), but the std deviation has gone down significantly. This implies that when using normalized data, the performance on the test set is much less dependent on how the data happened to be (randomly) split into a training and a test data set.

## C. Increate the number of epochs (5 marks)

Repeat Part B __but use 100 epochs this time for training__.

In [30]:
mean_mse, std_mse = evaluate_model(regression_model, predictors_norm, target, epochs=100)

# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models on normalized data, trained 100 epochs: mean = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models on normalized data, trained 100 epochs: mean = 153.001, std dev = 17.042


__How does the mean of the mean squared errors compare to that from Step B?__

The mean squared error on the test set more than halved. Clearly, there was plenty of improvement to be gained with additional training. The standard deviation has gone down a lot as well.

## D. Increase the number of hidden layers.

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [32]:
def regression_model_D():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [33]:
# Evaluate model D
mean_mse, std_mse = evaluate_model(regression_model_D, predictors_norm, target)

# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models with 3 hidden layers: mean = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models with 3 hidden layers: mean = 123.540, std dev = 15.873


__How does the mean of the mean squared errors compare to that from Step B?__

The mean of the mean squared errors is a lot better than in part B (123 vs 314). This shows that the network with multiple hidden layers was significantly better at learning to predict the 'Strength' feature than the single layer network.

## Congratulations!
__From now you are able to create Regression Models with Keras as well as you able to perform some Deep Learning probelems end to end using Keras.__